In [2]:
import pandas as pd
from pathlib import Path

In [3]:
data = Path('../data')

In [6]:
list(data.glob('*.csv'))

[WindowsPath('../data/customers.csv'),
 WindowsPath('../data/orders.csv'),
 WindowsPath('../data/products.csv'),
 WindowsPath('../data/sales.csv')]

In [7]:
df_sales = pd.read_csv(data / 'sales.csv')

In [8]:
df_sales.head()

,sales_id,order_id,product_id,price_per_unit,quantity,total_price
0,0,1,218,106,2,212
1,1,1,481,118,1,118
2,2,1,2,96,3,288
3,3,1,1002,106,2,212
4,4,1,691,113,3,339


In [9]:
df_orders = pd.read_csv(data / 'orders.csv')

In [10]:
df_orders.head()

,order_id,customer_id,payment,order_date,delivery_date
0,1,64,30811,2021-8-30,2021-09-24
1,2,473,50490,2021-2-3,2021-02-13
2,3,774,46763,2021-10-8,2021-11-03
3,4,433,39782,2021-5-6,2021-05-19
4,5,441,14719,2021-3-23,2021-03-24
